In [1]:
import numpy as np
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import molmass

In [23]:
# reading_data
data_wb = xlrd.open_workbook('CHE223-Assignment1 for 2023-data sets up to 2019.xls')

# FInding Columns of Ions and Postal Codes
ion_dict = {}
postal_code = {}

for sheet in data_wb.sheets():

    year = ''
    for char in sheet.name:
        if char.isdigit(): year += char
    year = int(year)

    ions = []
    for row in np.arange(11):
        for col in np.arange(sheet.ncols):
            value = sheet.cell_value(row,col)

            if type(value) is str:
                if len(value)<=3:
                    try :
                        molmass.Formula(value)._elements
                        ions.append((year, col, row, value))
                    except: pass
                elif 'postal' in value.lower():
                    postal_code[sheet] = col

    ion_dict[sheet] = ions
    

In [86]:
# iterate through rows and record values based on (column, ion) pairs

data = []

for sheet in data_wb.sheets():
    for i in np.arange(len(ion_dict[sheet])):

        ion_tuple = ion_dict[sheet][i]
        for row in np.arange(ion_dict[sheet][i][2]+1, sheet.nrows):

            value = sheet.cell_value(row,ion_dict[sheet][i][1])

            # Data cleaning
            if type(value) is str:
                if 'n' in value.lower() or '-' in value or value.strip() == '':
                    value = np.nan
                
                elif value.endswith('ppm'):
                    value = float(value.strip('ppm'))

                elif '..' in value:
                    value = float(value.replace('..','.'))

                elif '. ' in value:
                    value = float(value.replace('. ','.'))
                
                elif '^' in value:
                    value = float(value[0:4]) * 10 ** int(value[-1])

            if sheet in postal_code:
                pocode = sheet.cell_value(row, postal_code[sheet]).strip().upper()[0:3]
                
                if pocode.strip() == '':
                    pocode = np.nan

            else:
                pocode = np.nan
            
            data.append([ion_tuple[0], ion_tuple[3], value, pocode])



df = pd.DataFrame(data, columns=['Year', 'Ion', 'Conc (mg/L)', 'Postal Code'])